In my analysis, I didn't have enough time to compute the complaints within .1 miles for my random points, but this code would do it.

In [1]:
import pandas as pd
from scipy.spatial import distance
from math import sin, cos, sqrt, atan2, radians
import time

permits = pd.read_csv('data/fivek_random_for_nypd.csv')
nypd = pd.read_csv('../capstone_large_data_sets/nypd_min.csv')

def get_distance(point1, point2):
    R = 4182 #miles
    lat1 = radians(point1[0])
    lon1 = radians(point1[1])
    lat2 = radians(point2[0])
    lon2 = radians(point2[1])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    distance = R * c
    return distance


# note: the latitude and longitude I'm using are in decimals NOT minutes, so I do the same here
# these are set at a tenth of a mile to start, with 1 degree late = 69 miles and 1 degree longitude = 40 miles
#max_delt_lat = 1/690
#max_delt_long = 1/400
#hard coded to save time


t0 = time.time()
for i in permits.index:
    t1 = time.time()
    print(f'Index {i} started')
    lat = permits.loc[i, 'latitude']
    long = permits.loc[i, 'longitude']
    point1 = [lat, long]
    complaint_count = 0
    for n in nypd.index:
        #if permits.loc[i, 'borough'] == nypd.loc[n, 'boro_nm']:
        n_lat = nypd.loc[n, 'latitude']
        n_long = nypd.loc[n, 'longitude']
        if (abs(lat - n_lat)<= 1/690) or (abs(lat - n_long)<= 1/400):
            point2 = [n_lat, n_long]
            distance = get_distance(point1, point2)
            if distance <= .1:
                complaint_count += 1
    permits.loc[i, 'total_complaints'] = complaint_count
    print(f"For index {i}, the complaint count was {complaint_count}")
    print(f"Permit {i} finished at {time.time()}, after {time.time()-t1}, {time.time()-t0} since the start of the code.")

permits.to_csv('data/fivek_random_points_nypd.csv')
print('')
print("Time to run", time.time()-t0)

Index 0 started
For index 0, the complaint count was 0
Permit 0 finished at 1660559675.0295072, after 84.79790472984314, 84.7990210056305 since the start of the code.
Index 1 started
For index 1, the complaint count was 706
Permit 1 finished at 1660559760.7200189, after 85.69047927856445, 170.48952507972717 since the start of the code.
Index 2 started
For index 2, the complaint count was 555
Permit 2 finished at 1660559846.240588, after 85.52055883407593, 256.0100939273834 since the start of the code.
Index 3 started


KeyboardInterrupt: 